# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [1]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [2]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 998
    X = torch.zeros(len(usage_list),max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.full((len(usage_list),max_len),-1,dtype=torch.long)
    lens = []

    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[i,:cur_len, :] = feat
        if mode == 'train':
          y[i,:cur_len] = label
        lens.append(cur_len)

    

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y,lens
    else:
      return X,lens


# Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None,lens = None):
        self.data = X
        self.lens = lens
        if y is not None:
            self.label = y
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx], self.lens[idx]
        else:
            return self.data[idx],self.lens[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [4]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        # self.fc = nn.Sequential(
        #     BasicBlock(input_dim, hidden_dim),
        #     *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
        #     nn.Linear(hidden_dim, output_dim)
        # )
        self.lstm = nn.LSTM(
            input_size = input_dim,
            hidden_size = hidden_dim,
            num_layers = hidden_layers,
            dropout = drop,
            bidirectional = True,
            batch_first = True    
        )
        self.cls = nn.Sequential(
            nn.ReLU(),
            nn.Linear(2*hidden_dim, output_dim)
        )
        self.dropout = nn.Dropout(drop)
        self.ln = nn.LayerNorm(input_dim)
        self.ln2 = nn.LayerNorm(2*hidden_dim)
    def forward(self, x):
        # x = self.fc(x)
        x = self.ln(x)
        x, _ = self.lstm(x)
        x = self.ln2(x)
        x = self.dropout(x)
        x = self.cls(x)
        return x

# Hyper-parameters

In [5]:
concat_nframes = 3
train_ratio = 0.95


seed = 11922150     
batch_size = 8        
num_epoch = 200     
   
learning_rate = 1e-3    
model_path = './model.ckpt'  
early_stop = 20
drop = 0.5

input_dim = 39 * concat_nframes  
hidden_layers = 5        
hidden_dim = 128      


# Dataloader

In [6]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y,train_lens = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y,val_lens = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y,train_lens)
val_set = LibriDataset(val_X, val_y,val_lens)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3257


3257it [00:00, 6835.73it/s]


[INFO] train set
torch.Size([3257, 998, 117])
torch.Size([3257, 998])
[Dataset] - # phone classes: 41, number of utterances for val: 172


172it [00:00, 6664.45it/s]

[INFO] val set
torch.Size([172, 998, 117])
torch.Size([172, 998])


# Training

In [ ]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss(ignore_index = -1) 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay = 0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 
                mode='min', 
                factor=0.1, 
                patience=5,
                eps = 1e-4,
                verbose=True
            )

best_acc = 0.0
stop_count = 0
train_count = sum(map(lambda x:sum(x[2]),train_loader))
val_count = sum(map(lambda x:sum(x[2]),val_loader))
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels, lens = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() 
        outputs = model(features)
        loss = criterion(outputs.permute(0,2,1),labels)
        loss.backward()
        train_loss += loss.item()
        pred = torch.argmax(outputs,dim = -1)
        train_acc += (pred.detach() == labels.detach()).sum().item()
#         for idx, l in enumerate(lens):
#             loss = criterion(outputs[idx][:l], labels[idx][:l])
#             loss.backward(retain_graph=True) 
#             train_loss += loss.item()
#             pred = torch.argmax(outputs[idx][:l],dim = -1)
#             
#             train_count += l
        optimizer.step() 



    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels,lens = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            loss = criterion(outputs.permute(0,2,1),labels)
            val_loss += loss.item()
            pred = torch.argmax(outputs,dim = -1)
            val_acc += (pred.detach() == labels.detach()).sum().item()
#             for idx, l in enumerate(lens):
#                 loss = criterion(outputs[idx][:l], labels[idx][:l])
#                 val_loss += loss.item()
#                 pred = torch.argmax(outputs[idx][:l],dim = -1)
#                 val_acc += (pred.detach() == labels[idx][:l].detach()).sum().item()
#                 val_count += l
    scheduler.step(val_loss)
    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/train_count} Loss: {train_loss/len(train_set)} | Val Acc: {val_acc/val_count} loss: {val_loss/len(val_set)}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc + 0.0001:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/val_count:.5f}')
        stop_count = 0
    else:
        stop_count += 1
    if stop_count >= early_stop:
        print(f"Early stop at {epoch} epoch")
        break

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 19.94it/s]


[001/200] Train Acc: 0.5324382185935974 Loss: 0.2018808119861414 | Val Acc: 0.6549369096755981 loss: 0.14704620664895968
saving model with acc 0.65494


 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 387/408 [01:20<00:04,  4.78it/s]

In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load data
test_X,test_lens = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None, test_lens)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

Ensemble

In [ ]:
#ensemble
# test_X,test_lens = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=3)
# test_set = LibriDataset(test_X, None, test_lens)
# test_loader2 = DataLoader(test_set, batch_size=batch_size, shuffle=False)
# test_loader2 = [batch[0] for batch in test_loader2 ]
# model2 = Classifier(input_dim=3*39, hidden_layers=5, hidden_dim=128).to(device)
# model2.load_state_dict(torch.load('./831.ckpt'))

In [ ]:
preds = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features,lens = batch
        features = features.to(device)
        
        outputs = model(features)
#         outputs = outputs + model2(features) #ensemble
        for idx, l in enumerate(lens):
            pred = torch.argmax(outputs[idx][:l],dim = -1).cpu().numpy()
            preds = np.append(preds,pred)
            


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(preds):
        f.write('{},{}\n'.format(i, y))